<a href="https://colab.research.google.com/github/smurflucu/pub/blob/main/automailer_settlement_M001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.cloud import bigquery
import pandas as pd
from smtplib import SMTP
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders
import os
from datetime import datetime

# Set up Google Cloud credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/content/files/control/credential.json'
client = bigquery.Client()

# Format the current date as a string in YYYY-MM-DD format
current_date_str = datetime.now().strftime('%Y-%m-%d')
#current_date_str = '2024-08-20'
current_state = 'M001'

output_folder = '/content/files/excel'
os.makedirs(output_folder, exist_ok=True)

# Query to get email details
query_email = f"""
SELECT id_email,email_to,email_cc,email_bcc,subject,body_header,body_mid,body_footer,body_sender,merchant_name,filename,file_format,tbl_body,mod,id_search,status1,status2,note
FROM `merchant.load_list_email_tmp`
WHERE status1 = 'Active' and status2 = '{current_state}'
ORDER BY id_email
"""
email_df = client.query(query_email).to_dataframe()

# Print the email data
print(email_df.head())

# Function to generate Excel file with two sheets
def generate_excel(id_search, mod, filename, current_date_str):
    query_sheet1 = f"""
    CALL merchant.proc_mm_read('{current_date_str}', '{current_date_str}', '{mod}', 'q_summary', '{id_search}')
    """
    df_sheet1 = client.query(query_sheet1).to_dataframe()

    query_sheet2 = f"""
    CALL merchant.proc_mm_read('{current_date_str}', '{current_date_str}', '{mod}', 'q_detail', '{id_search}')
    """
    df_sheet2 = client.query(query_sheet2).to_dataframe()

    filename_with_path = os.path.join(output_folder, filename)
    if not df_sheet1.empty and not df_sheet2.empty:
        with pd.ExcelWriter(filename_with_path) as writer:
            df_sheet1.to_excel(writer, sheet_name='summary', index=False)
            df_sheet2.to_excel(writer, sheet_name='detail', index=False)
        return filename_with_path, df_sheet1  # Return the DataFrame for table generation
    else:
        return None, None

# Function to convert DataFrame to an HTML table with right alignment for columns 5-10
def df_to_html(df):
    table_html = '<table border="1" cellpadding="5" cellspacing="0" style="border-collapse: collapse;">'

    # Add table header with yellow background
    table_html += '<tr style="background-color: yellow;">'
    for column in df.columns:
        table_html += f'<th>{column}</th>'
    table_html += '</tr>'

    # Add table rows with specific alignment for columns 5-10
    for index, row in df.iterrows():
        table_html += '<tr>'
        for i, cell in enumerate(row):
            if 4 <= i <= 9:  # Columns 5 to 10 (0-based index)
                table_html += f'<td style="text-align: right;">{int(cell):,}</td>'  # Right align and format as integer
            else:
                table_html += f'<td>{cell}</td>'
        table_html += '</tr>'

    table_html += '</table>'
    return table_html

# Function to send email with attachment and include the table
def send_email(to, cc, bcc, subject, body, header, footer, sender, filename, table_html):
    smtp_server = 'smtp.gmail.com'
    smtp_port = 587
    smtp_user = 'mis.report@ottodigital.id'
    smtp_password = 'iibp wycj jtda idrs'  # Replace with your email password

    msg = MIMEMultipart()
    msg['From'] = smtp_user
    msg['To'] = to
    msg['Cc'] = cc
    msg['Bcc'] = bcc
    msg['Subject'] = subject

    # Combine header, body, table, and footer into a single HTML body
    full_body = f"<b>{header}</b><br><br>{body}<br><br>{table_html}<br><br>{footer}<br><b>{sender}</b>"
    msg.attach(MIMEText(full_body, 'html'))  # Set the email body as HTML

    if filename:
        with open(filename, 'rb') as attachment:
            part = MIMEBase('application', 'octet-stream')
            part.set_payload(attachment.read())
            encoders.encode_base64(part)
            part.add_header(
                'Content-Disposition',
                f'attachment; filename={os.path.basename(filename)}',
            )
            msg.attach(part)

        with SMTP(smtp_server, smtp_port) as server:
            server.starttls()
            server.login(smtp_user, smtp_password)
            server.send_message(msg)

        return True
    else:
        print(f"No data found for {filename}. Email not sent.")
        return False

# Log files
log_file_path = os.path.join(output_folder, 'email_log_' + current_state + '.txt')

# Generate Excel files and send emails
for index, row in email_df.iterrows():
    id_search = row['id_search']
    mod = row['mod']
    filename = row['filename'] + ' ' + current_date_str + '.xlsx'

    # Generate the Excel file and get the summary table DataFrame
    file_path, df_sheet1 = generate_excel(id_search, mod, filename, current_date_str)

    # Convert the DataFrame to an HTML table
    table_html = df_to_html(df_sheet1) if df_sheet1 is not None else ""

    # Send the email
    success = send_email(
        to=row['email_to'],
        cc=row['email_cc'],
        bcc=row['email_bcc'],
        subject=row['subject'],
        body=row['body_mid'],
        header=row['body_header'],
        footer=row['body_footer'],
        sender=row['body_sender'],
        filename=file_path,
        table_html=table_html
    )

    # Log the result
    with open(log_file_path, 'a') as log_file:
        if success:
            log_file.write(f"{datetime.now()}: Email sent successfully for id_search {id_search}\n")
        else:
            log_file.write(f"{datetime.now()}: Failed to send email for id_search {id_search}\n")

    print(f"Processed email for id_search {id_search}")

print(f"Process completed. Log file created at {log_file_path}")


  id_email                   email_to email_cc email_bcc  \
0   E00386  mis.report@ottodigital.id     None      None   
1   E00387  mis.report@ottodigital.id     None      None   
2   E00388  mis.report@ottodigital.id     None      None   

                                          subject body_header  \
0  [TEST] Laporan Settlement GREEN LAKE CITY  GLC   Dear Team   
1  [TEST] Laporan Settlement PANTAI INDAH KAPUK 2   Dear Team   
2    [TEST] Laporan Settlement SEDAYU SQUARE  SSQ   Dear Team   

                                            body_mid  \
0  Berikut terlampir laporan settlement harian. <...   
1  Berikut terlampir laporan settlement harian. <...   
2  Berikut terlampir laporan settlement harian. <...   

                                         body_footer  \
0  Untuk rincian transaksi yang lebih lengkap, si...   
1  Untuk rincian transaksi yang lebih lengkap, si...   
2  Untuk rincian transaksi yang lebih lengkap, si...   

                                         body_se